In [1]:
import os
os.chdir('/home/cjtu/projects/essi21/essi21')
import numpy as np
import pandas as pd
import mixing as mm
import matplotlib.pyplot as plt

validation = pd.read_csv('/home/cjtu/projects/essi21/data/haworth_ejecta_thickness.csv', skiprows=7, usecols=[0, 7, 8], names=['cname', 'dist', 'thickness'])

df = mm.read_crater_list()
dist = mm.get_ejecta_distances(df)
thickness = mm.get_ejecta_thickness(dist, df.rad.values)
df = df.set_index('cname')


In [35]:
ej = pd.read_csv('/home/cjtu/projects/essi21/data/210624/cannon00000/ej_columns_cannon00000.csv')
haworth = ej.iloc[6:7]
new_ej = ej[ej.Haworth !=0]
out = pd.concat([haworth, new_ej], ignore_index=True)
out.to_csv('/home/cjtu/projects/essi21/data/210624/cannon00000/ej_thickness_squashed.csv')

In [3]:
grd_x =  mm._GRD_X
grd_y =  mm._GRD_Y
cx, cy = df.loc['Faustini', ['x', 'y']]
ci = np.where(df.index == 'Faustini')[0][0]
c_dist = dist[:, :, ci]
c_thick = thickness[:, :, ci]

cxi, cyi = mm.get_grd_ind(cx, cy, grd_x, grd_y)
cdist = mm.grid_interp(cx, cy, c_dist)
cthick = mm.grid_interp(cx, cy, c_thick)
print(cx, cy)
print(cxi, cyi, ci)
print(grd_x[0, cxi], grd_y[cyi, 0])
print(cdist, cthick)

84432.50510944115 8427.484384640939
484 392 2
84000.0 8000.0
nan 0.0


In [4]:
hx, hy = df.loc['Haworth', ['x', 'y']]
hdist = mm.grid_interp(hx, hy, c_dist)
hthick = mm.grid_interp(hx, hy, c_thick)
print(hdist, hthick)

113260.28852133433 1.472943092159688


In [5]:
def grid_interp(x, y, grdvalues, grd_x=mm._GRD_X, grd_y=mm._GRD_Y):
    """Return ejecta thickness at (x, y) in ejecta_thickness 2D grid."""
    ix, iy = mm.get_grd_ind(x, y, grd_x, grd_y)
    gx, gy = grd_x.flatten(), grd_y.flatten()

    dy = (y - gy[iy]) / (gy[iy + 1] - gy[iy])
    interp_y = (1 - dy) * grdvalues[iy] + dy * grdvalues[iy + 1]

    dx = (x - gx[ix]) / (gx[ix + 1] - gx[ix])
    interp = (1 - dx) * interp_y[ix] + dx * interp_y[ix + 1]
    return interp

In [6]:
df = mm.read_crater_list()
def get_crater_distances(df):
    """
    Return 2D array of great circle dist between all craters in df.
    """
    out = np.zeros((len(df), len(df)), dtype=mm._DTYPEF)
    for i in range(len(df)):
        for j in range(i):
            d = mm.gc_dist(*df.iloc[i][['lon', 'lat']], *df.iloc[j][['lon', 'lat']])
            out[i, j] = d
    return out
d = mm.get_crater_distances(df.iloc[:4])
print(d)
print(list(zip(df.cname.values, d[:,0])))

[[       nan        nan        nan        nan]
 [ 59815.996        nan        nan        nan]
 [113282.875  53466.875        nan        nan]
 [183806.84  128677.336  84923.           nan]]
[('Haworth', nan), ('Shoemaker', 59815.996), ('Faustini', 113282.875), ('Amundsen', 183806.84)]


In [16]:
ej_thick_matrix = mm.get_ejecta_thickness_matrix(df, ej_distances)

In [25]:
time_arr = mm._TIME_ARR
ej_distances = mm.get_crater_distances(df, True)
rad = df.rad.values[:, np.newaxis]
ej_thick = mm.get_ejecta_thickness(ej_distances, rad)
time_idx = np.searchsorted(-time_arr, -df.age.values)
ny, nx = ej_thick.shape[:2]
ej_thick_time2 = np.zeros((len(time_arr), len(time_idx)), dtype=mm._DTYPE)
for i, t_idx in enumerate(time_idx):
    ej_thick_time2[t_idx, :] += ej_thick[:, i]
            # ej_thick_time[time_idx, i] += ej_thick[i, j]

In [34]:
np.where(df.cname.isin(mm.COLD_TRAP_CRATERS))

(array([ 0,  1,  2,  3,  5,  6,  8, 14, 21, 22]),)

In [24]:
time_arr = mm._TIME_ARR
ej_distances = mm.get_crater_distances(df, True)
rad = df.rad.values
ej_thick = mm.get_ejecta_thickness(ej_distances, rad)
time_idx = np.searchsorted(-time_arr, -df.age.values)
ny, nx = ej_thick.shape[:2]
ej_thick_time = np.zeros((len(time_arr), len(time_idx)), dtype=mm._DTYPE)
for i, t_idx in enumerate(time_idx):
    ej_thick_time[t_idx, :] += ej_thick[:, i]
            # ej_thick_time[time_idx, i] += ej_thick[i, j]

In [8]:
ej_thick2 = np.zeros((len(time_arr), len(time_idx)), dtype=mm._DTYPE)
i_idx = np.arange(len(time_idx))
ej_thick2[time_idx, :] += ej_thick[:, i_idx]
# r = df.rad.values
# print(list(zip(df.cname.values, mm.get_ejecta_thickness(d, r)[:,0])))
# len(time_idx)

In [26]:
np.testing.assert_array_equal(ej_thick_time, ej_thick_time2)

AssertionError: 
Arrays are not equal

Mismatched elements: 280 / 10200 (2.75%)
Max absolute difference: 610.3255
Max relative difference: 34.96499
 x: array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],...
 y: array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],...

In [41]:
ej_thick[:, 0]

array([        nan, 20.96124999,  1.47216912,  9.58291595, 65.3848693 ,
        1.05698854, 17.14572192,  0.99747627,  0.42675695,  1.0518162 ,
       12.31031549,  9.3396016 ,  0.6378449 ,  2.35493715,  5.49103836,
        0.13963879,  1.00796395,  0.10840974,  0.2664189 ,  1.44916295,
        0.35874482,  1.20474503,  2.97617481,  0.12375152])

In [19]:
np.testing.assert_array_almost_equal(ej_thick_matrix, ej_thick_time)

In [17]:
ejdf = pd.DataFrame(ej_thick_matrix, columns=df.cname.values)
ejdf.head(17)

,Haworth,Shoemaker,Faustini,Amundsen,Cabeus,Cabeus B,de Gerlache,Hedervari,Idel'son L,Unnamed 1,...,Sverdrup,Wiechert P,Unnamed 2,Wiechert,Idel'son,Unnamed 3,Wiechert U,Wiechert J,Shackleton,Amundsen C
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,20.961250,1.472169,9.582916,65.384872,1.056989,17.145721,0.997476,0.426757,1.051816,...,5.491038,0.139639,1.007964,0.108410,0.266419,1.449163,0.358745,1.204745,2.976175,0.123752
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
